In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
data

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
import seaborn as sns
sns.heatmap(data.isnull())

In [ ]:
from sklearn import preprocessing
Lbe = preprocessing.LabelEncoder()
data['PassengerId'] = Lbe.fit_transform(data['PassengerId'])
data['PassengerId']

In [ ]:
data.head()

In [ ]:
data.HomePlanet = data.HomePlanet.fillna('0')
data.CryoSleep  = data.CryoSleep .fillna(0)
data.Cabin      = data.Cabin.fillna(0)
data.Destination = data.Destination.fillna('0')
data.Age = data.Age.fillna(0)
data.VIP = data.VIP.fillna(0)
data.RoomService = data.RoomService.fillna(0)
data.FoodCourt = data.FoodCourt.fillna(0)
data.ShoppingMall = data.ShoppingMall.fillna(0)
data.Spa = data.Spa.fillna(0)
data.VRDeck = data.VRDeck.fillna(0)
data.Name = data.Name.fillna('0')

In [ ]:
data.head()

In [ ]:
import pandas as pd
data['RoomService'] = data['RoomService'].astype(int)
data['Age'] = data['Age'].astype(int)
data['FoodCourt'] = data['FoodCourt'].astype(int)
data['ShoppingMall'] = data['ShoppingMall'].astype(int)
data['Spa'] = data['Spa'].astype(int)
data['VRDeck'] = data['VRDeck'].astype(int)

In [ ]:
data.Destination = data.Destination.replace({'TRAPPIST-1e':'TRAPPIST','55 Cancri e':'Cancri','PSO J318.5-22':'PSO'})
data['Transported']= data['Transported'].replace(True,1).replace(False,0)

In [ ]:
x = data.iloc[:,0:13]

In [ ]:
x

In [ ]:
x = data.drop(columns = ['Cabin','Name','Transported'])

In [ ]:
y = data['Transported']
y

In [ ]:
data.dtypes

In [ ]:
x.head()

In [ ]:
x.iloc[:,3].unique()

In [ ]:
nom = [1,3,2,5]
#ordi = [2,5]
num = [0,6,7,8,9,10]
bina = [4]

In [ ]:
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder,PowerTransformer,Binarizer
from sklearn.compose import make_column_transformer
from sklearn import set_config
trans = make_column_transformer((OneHotEncoder(sparse = False),nom),
                                #(OrdinalEncoder(),ordi),
                                (PowerTransformer(),num),
                                (Binarizer(),bina),
                                 remainder = 'passthrough')
set_config(display = 'diagram')
trans      

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.25)

In [ ]:
import seaborn as sns
sns.countplot(y)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
lr = LogisticRegression(solver = 'liblinear')
pipe = make_pipeline(trans,lr)
pipe

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
pred = pipe.predict(x_test)
pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(pred,y_test)*100

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(pipe,x_test,y_test)

In [ ]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
Over =RandomOverSampler()
o_x,o_y=Over.fit_resample(x,y)
o_y.value_counts()

In [ ]:
from imblearn.pipeline import make_pipeline
S = SMOTE()
pipe_s = make_pipeline(trans,S,lr)

In [ ]:
pipe_s.fit(x_train,y_train)

In [ ]:
pred_s = pipe.predict(x_test)
pred_s

In [ ]:
accuracy_score(pred_s,y_test)*100

In [ ]:
from sklearn.svm import SVC
algorithm = SVC()
pipe_a = make_pipeline(trans,S,algorithm)

In [ ]:
pipe_a.fit(x_train,y_train)

In [ ]:
pred_a =pipe.predict(x_test)
pred_a

In [ ]:
accuracy_score(pred_a,y_test)*100

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(n_estimators = 300,min_samples_leaf = 4) 
pipe_GB = make_pipeline(trans,S,GB)

In [ ]:
pipe_GB.fit(x_train,y_train)
pred_gb = pipe_GB.predict(x_test)
accuracy_score(pred_gb,y_test)*100

In [ ]:
from sklearn.ensemble import RandomForestClassifier
F = RandomForestClassifier(n_estimators = 100,criterion = 'entropy')
pipe_F = make_pipeline(trans,S,F)

In [ ]:
pipe_F.fit(x_train,y_train)
pred_F = pipe_F.predict(x_test)
accuracy_score(pred_F,y_test)*100

In [ ]:
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt

plot_roc_curve(pipe_a,x_test,y_test)

plt.plot([0,1],[0,1],"k--")                                                                # k = color of the diagnol 

plt.xlabel("TPR")
plt.ylabel("FPR")
plt.show()

In [ ]:
test_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test_data

In [ ]:
test_data.iloc[:,3].unique()

In [ ]:
test_data.Destination = test_data.Destination.replace({'TRAPPIST-1e':'TRAPPIST','55 Cancri e':'Cancri','PSO J318.5-22':'PSO'})

In [ ]:
test_data

In [ ]:
test_data.isnull().sum()

In [ ]:
test_data.HomePlanet = test_data.HomePlanet.fillna('0')
test_data.CryoSleep  = test_data.CryoSleep .fillna(0)
test_data.Cabin      = test_data.Cabin.fillna(0)
test_data.Destination = test_data.Destination.fillna('0')
test_data.Age = test_data.Age.fillna(0)
test_data.VIP = test_data.VIP.fillna(0)
test_data.RoomService = test_data.RoomService.fillna(0)
test_data.FoodCourt = test_data.FoodCourt.fillna(0)
test_data.ShoppingMall = test_data.ShoppingMall.fillna(0)
test_data.Spa = test_data.Spa.fillna(0)
test_data.VRDeck = test_data.VRDeck.fillna(0)
test_data.Name = test_data.Name.fillna('0')

In [ ]:
import pandas as pd
test_data['RoomService'] = test_data['RoomService'].astype(int)
test_data['Age'] = test_data['Age'].astype(int)
test_data['FoodCourt'] = test_data['FoodCourt'].astype(int)
test_data['ShoppingMall'] = test_data['ShoppingMall'].astype(int)
test_data['Spa'] = test_data['Spa'].astype(int)
test_data['VRDeck'] = test_data['VRDeck'].astype(int)

In [ ]:
x1 = test_data.drop(columns = ['Cabin','Name'])

In [ ]:
from sklearn import preprocessing
Lb = preprocessing.LabelEncoder()
x1['PassengerId'] = Lb.fit_transform(x1['PassengerId'])
x1['PassengerId']

In [ ]:
x1.iloc[:,5].unique()

In [ ]:
nom = [1,3,2,5]
#ordi = [2,5]
num = [0,6,7,8,9,10]
bina = [4]

In [ ]:
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder,PowerTransformer,Binarizer
from sklearn.compose import make_column_transformer
from sklearn import set_config
trans = make_column_transformer((OneHotEncoder(sparse = False),nom),
                                #(OrdinalEncoder(),ordi),
                                (PowerTransformer(),num),
                                (Binarizer(),bina),
                                 remainder = 'passthrough')
set_config(display = 'diagram')
trans      

In [ ]:
pred_testdata = pipe_GB.predict(test_data)
pred_testdata

In [ ]:
submit = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
submit

In [ ]:
import pandas as pd
submit = pd.DataFrame({'PassengerId':test_data['PassengerId'],'Transported':pred_testdata.astype("bool")})
submit.to_csv('submission.csv',index = False)

In [ ]:
submit.Transported.value_counts()

In [ ]:
test_data.PassengerId.unique

In [ ]:
submit.head()

In [ ]:
y_test.head()